In [63]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import sklearn 


In [107]:
#LOAD GIS WATERMAIN AND BREAK DATA
df = pd.read_csv('Kirk_All.csv')
df.dropna(inplace=True) # Are there any columns where we can deal with NA values?
# Can we use imputation instead?
breaks = pd.read_csv('Kirk_Break_Data.csv')
df['INSTYEAR'] = df['INSTYEAR'].map(lambda x: int((str(x)[-12:-8])))
df


,FID,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
1,1,2,FIRE,2.0,2.0,UNK,1992,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
3,3,4,FIRE,3.0,6.0,DI,2005,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,4,5,FIRE,149.0,8.0,AC,1963,Ur,7000,6.0,,,
5,5,5,FIRE,149.0,8.0,AC,1963,Ur,7000,3.0,S I 405ONRAMP,25,RAMP
6,6,5,FIRE,149.0,8.0,AC,1963,Ur,7000,6.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41815,41815,3962,DIST,4.0,8.0,DI,1977,No,9000,0.0,120TH AVE NE,25,COLLECTOR
41816,41816,3963,DIST,1.0,12.0,DI,1971,No,9000,0.0,120TH AVE NE,25,COLLECTOR
41817,41817,5622,DIST,1.0,8.0,DI,2003,AgC,10000,2.0,117TH AVE NE,25,NEIGHBORHOOD ACCESS
41818,41818,5623,DIST,1.0,8.0,DI,2003,AgC,10000,2.0,117TH AVE NE,25,NEIGHBORHOOD ACCESS


In [77]:
breaks

,Date,Location,Zone,Size,Type,Comment,Year,Install_Year
0,1/8,347 7th Ave,315,6,CI,NaN,2004,NaN
1,1/9,696 16th Ave W,315,8,AC,NaN,2004,NaN
2,1/21,740 16th Ave W,315,6,AC,NaN,2004,NaN
3,2/25,10132 NE 59th,285,8,AC,Leak,2004,NaN
4,3/16,18th Ave & 6th St.,450,12,AC,Leak,2004,NaN
...,...,...,...,...,...,...,...,...
63,10/31/17,10830 NE 68th St,450,8,AC,Full Break,2017,NaN
64,5/23/18,11828 NE 75th St,650,6,AC,Bead Break,2018,NaN
65,5/30/19,11828 NE 75th St,650,6,AC,Break at abandoned service tap,2019,NaN
66,6/3/19,10425 NE 58th St,450,6,AC,Full Break,2019,NaN


In [108]:
# GROUP PIPES BY THEIR UNIQUE ID - NEEDED BECAUSE OF HOW SLOPE BREAKS UP PIPES IN DIFFERENT SLOPE CLASSES
grouping = df.groupby('FID_WA_Mai').groups
unique_pipes = df['FID_WA_Mai'].unique()
indices = []
for i in range(len(unique_pipes)):
    pipe = unique_pipes[i]
    val = grouping[pipe][0]
    indices.append(val)
    

In [109]:
# CALCULATE AVERAGE SLOPES FOR PIPES THAT GOT SPLIT INTO MULTIPLE ROWS
slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
slopes = pd.Series(slopes)


In [141]:
# REMOVE SLP_CLASS COLUMN, ADD AVERAGE SLOPE SEREIS FOR EACH UNIQUE PIPE
filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
filtered = filtered.drop('SLP_CLASS', axis=1)
filtered['SLOPE'] = slopes.values

filtered


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
1,2,FIRE,2.0,2.0,UNK,1992,AgC,7000,122ND LN NE,25,NEIGHBORHOOD ACCESS,1.000000
3,4,FIRE,3.0,6.0,DI,2005,Ur,4000,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL,0.000000
4,5,FIRE,149.0,8.0,AC,1963,Ur,7000,,,,3.888889
22,6,FIRE,1.0,12.0,DI,1978,Ur,6000,120TH AVE NE,25,MINOR ARTERIAL,1.000000
23,7,FIRE,1.0,6.0,DI,2002,AmC,13000,1ST ST,25,NEIGHBORHOOD ACCESS,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
41801,7742,DIST,15.0,8.0,DI,2017,AgD,5000,96TH AVE NE,25,NEIGHBORHOOD ACCESS,0.785714
41804,7743,DIST,296.0,8.0,AC,1943,AgC,9000,NE 68TH ST,25,MINOR ARTERIAL,4.000000
41816,3963,DIST,1.0,12.0,DI,1971,No,9000,120TH AVE NE,25,COLLECTOR,4.000000
41817,5622,DIST,1.0,8.0,DI,2003,AgC,10000,117TH AVE NE,25,NEIGHBORHOOD ACCESS,3.000000


In [154]:
filtered[filtered['LENGTH'] > 10.0]

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
4,5,FIRE,149.0,8.0,AC,1963,Ur,7000,,,,3.888889
29,13,FIRE,12.0,4.0,UNK,1997,KpB,7000,NE 62ND ST,25,NEIGHBORHOOD ACCESS,3.000000
75,53,FIRE,33.0,4.0,UNK,1991,AmC,13000,5TH AVE,25,NEIGHBORHOOD ACCESS,1.666667
105,79,FIRE,30.0,8.0,DI,1991,AgC,6000,CARILLON PT,10,NEIGHBORHOOD ACCESS,2.600000
129,99,FIRE,127.0,4.0,DI,2003,AgC,5000,113TH PL NE,10,NEIGHBORHOOD ACCESS,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
41778,7738,DIST,128.0,8.0,DI,2020,InC,11000,8TH ST S,25,COLLECTOR,1.428571
41785,7739,DIST,431.0,8.0,DI,2020,AgC,6000,NE 90TH ST,25,COLLECTOR,0.600000
41799,7740,DIST,12.0,8.0,DI,2001,AgD,5000,96TH AVE NE,25,NEIGHBORHOOD ACCESS,2.000000
41801,7742,DIST,15.0,8.0,DI,2017,AgD,5000,96TH AVE NE,25,NEIGHBORHOOD ACCESS,0.785714


In [135]:
break_1_8_04 = filtered[(filtered['FULL_STRNA'] == '7TH AVE') & (filtered['INSTYEAR'].equals(2003))]
break_1_8_04

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE


In [204]:
break_1_9_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W')] #& (filtered['INSTYEAR'] < 2005) & (filtered['INSTYEAR'] > 2003)]
break_1_9_04 = break_1_9_04[break_1_9_04['INSTYEAR'] == 2004]
break_1_9_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004) & (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 8.0)] 
nearby

similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [205]:
break_1_21_04 = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] == 2004)]
break_1_21_04

nearby = filtered[(filtered['FULL_STRNA'] == '16TH AVE W') & (filtered['INSTYEAR'] != 2004)] #& (filtered['Dist_Rail'] == 15000) & (filtered['DIAMETER'] == 6.0)] 


similar = filtered[(filtered['DIAMETER'] == 8.0) & (filtered['MATERIAL'] == 'AC') & (filtered['MUSYM'] == 'AmC')]
print(similar['FULL_STRNA'].unique())

similar = similar[(similar['FULL_STRNA'] == '17TH AVE W')]
similar

['10TH AVE' 'KIRKLAND CIR' '4TH ST' '17TH AVE W' '9TH AVE' '5TH AVE'
 '3RD ST' 'KIRKLAND WAY' '8TH LN' '15TH AVE' 'KIRKLAND AVE' 'NE 68TH ST'
 '19TH AVE' '8TH AVE' 'NE 45TH ST' ' ' '1ST ST' 'NE 44TH ST' 'NE 68TH PL'
 '6TH ST' '106TH AVE NE' '6TH AVE' '5TH ST' '7TH AVE' '109TH PL NE'
 '10TH AVE S' '5TH AVE S' 'LAKEVIEW DR' '5TH ST S' '6TH ST S' 'STATE ST S']


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
3851,709,DIST,34.0,8.0,AC,1968,AmC,14000,17TH AVE W,25,NEIGHBORHOOD ACCESS,0.5


In [197]:
break_2_25_04 = filtered[(filtered['FULL_STRNA'] == 'NE 59TH ST')]
break_2_25_04

# THIS WILL LIKELY NEED TO BE altered in data set

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
22011,3634,DIST,1.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
22012,3635,DIST,5.0,12.0,AC,1971,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,1.000000
39310,6927,DIST,261.0,8.0,AC,1976,KpB,7000,NE 59TH ST,25,NEIGHBORHOOD ACCESS,2.586207


In [ ]:
break_3_16_04 = 